In [1]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [2]:
############### Load the dataset ###################
path_train = 'all_train.csv'
train_data=pd.read_csv(path_train)
samples = train_data.sample(frac=0.1)

In [3]:
X =samples.iloc[:,1:]
Y =samples.iloc[:,0]
classes = 2
Y = to_categorical(Y,classes)

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(560000, 28) (560000, 2)
(140000, 28) (140000, 2)


In [5]:
s = MinMaxScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

In [7]:
n = X_train.shape[1]
dim = X_train.shape[1]
final_dim = min(32*32,n*n)
# convert 2-D to 3-D.
while dim<final_dim:
  features = int(dim/2)
  #f = feature-60
  model = Sequential()
  if (dim+features)>final_dim:
    features = final_dim - dim
    model.add(Dense(features,name='feature', activation='relu',input_shape=(dim,)))
  else:
    model.add(Dense(features,name='feature', activation='relu',input_shape=(dim,)))
  model.add(Dense(classes,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  model.fit(X_train,Y_train,epochs=1,batch_size=512,verbose=1)
  extract = Model(model.inputs, model.get_layer('feature').output)
  features1 = extract.predict(X_train)
  features2 = extract.predict(X_test)
  X_train = np.concatenate((X_train, features1), axis=1) # concatenate on horizontal axis
  X_test = np.concatenate((X_test, features2), axis=1)
  dim = X_train.shape[1]
print(X_train.shape,X_test.shape)
s = MinMaxScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

1094/1094 [==============================] - 2s 2ms/step - loss: 0.3033
(560000, 1024) (140000, 1024)


In [9]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(X_train)):
    img = Image.fromarray(X_train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(X_test)):
    img = Image.fromarray(X_test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(560000, 32, 32, 3) (140000, 32, 32, 3)


In [11]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/20
2188/2188 [==============================] - 161s 74ms/step - loss: 0.3098 - accuracy: 0.8555 - val_loss: 0.3152 - val_accuracy: 0.8507
Epoch 2/20
2188/2188 [==============================] - 158s 72ms/step - loss: 0.3054 - accuracy: 0.8570 - val_loss: 0.3078 - val_accuracy: 0.8565
Epoch 3/20
2188/2188 [==============================] - 163s 74ms/step - loss: 0.3093 - accuracy: 0.8561 - val_loss: 0.3184 - val_accuracy: 0.8507
Epoch 4/20
2188/2188 [==============================] - 164s 75ms/step - loss: 0.3063 - accuracy: 0.8566 - val_loss: 0.3089 - val_accuracy: 0.8542
Epoch 5/20
2188/2188 [==============================] - 166s 76ms/step - loss: 0.3046 - accuracy: 0.8572 - val_loss: 0.3068 - val_accuracy: 0.8563
Epoch 6/20
2188/2188 [==============================] - 166s 76ms/step - loss: 0.3037 - accuracy: 0.8575 - val_loss: 0.3062 - val_accuracy: 0.8568
Epoch 7/20
2188/2188 [==============================] - 166s 76ms/step - loss: 0.3033 - accuracy: 0.8578 - val_loss: 0

In [12]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

17500/17500 [==============================] - 160s 9ms/step - loss: 0.3005 - accuracy: 0.8578
training accuracy:  0.8577731847763062
testing accuracy:  0.8563214540481567
